<a href="https://colab.research.google.com/github/SorayaFrancoLopez/RN/blob/main/clasificar_emails.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [191]:
#Descargas de bibliotecas.
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from scipy.sparse import hstack
from google.colab import drive
from sklearn.metrics import accuracy_score

In [171]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [172]:
df = pd.read_csv("/content/drive/MyDrive/e-mails.csv", delimiter=';', encoding='latin1')

In [173]:
df

,emails,asunto,texto,prioridad
0,cliente1@email.com,Solicitud de Préstamo,"Hola, necesito solicitar un préstamo para una ...",Alta
1,cliente2@email.com,Consulta de Inversiones,"Estimado Gestor, ¿cuáles son las opciones de i...",Media
2,cliente3@email.com,Reclamo sobre Cargo,"He notado un cargo incorrecto en mi cuenta, ¿p...",Alta
3,cliente4@email.com,Pregunta sobre Cuenta de Ahorro,Quiero saber cuál es la tasa de interés actual...,Baja
4,cliente5@email.com,Actualización de Contacto,"Estimado Gestor, he cambiado mi dirección y nú...",Media
...,...,...,...,...
95,cliente96@email.com,Solicitud de Tarjeta de Crédito Empresarial,Quiero solicitar una tarjeta de crédito empres...,Alta
96,cliente97@email.com,Problema con Transferencia Bancaria,He tenido un problema con una transferencia ba...,Alta
97,cliente98@email.com,Informe de Estado de Cuenta,¿Puedes proporcionarme una copia de mi estado ...,Media
98,cliente99@email.com,Pregunta sobre Hipoteca,"Estimado Gestor, necesito información sobre la...",Baja


In [174]:
# Vectorización del campo 'texto'
text_vectorizer = TfidfVectorizer(max_features=50)
X_text = text_vectorizer.fit_transform(df['texto'])

# Vectorización del campo 'asunto'
subject_vectorizer = TfidfVectorizer(max_features=50)
X_subject = subject_vectorizer.fit_transform(df['asunto'])

# Combinar las dos representaciones TF-IDF
X_combined = hstack([X_text, X_subject])

In [175]:
dense_X_combined = X_combined.toarray()

In [177]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['prioridad'])

In [178]:
encoder = OneHotEncoder(sparse=False)
dense_X_combined_onehot = encoder.fit_transform(dense_X_combined)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [179]:
#division en train y test
X_train, X_test, y_train, y_test = train_test_split(dense_X_combined_onehot, y, test_size=0.2, random_state=42)

In [182]:
X_train

array([[1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.]])

In [183]:
X_test

array([[1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 0.],
       [1., 0., 0., ..., 0., 0., 1.],
       [1., 0., 0., ..., 0., 1., 0.]])

In [184]:
y_train

array([0, 2, 0, 0, 0, 1, 2, 0, 0, 0, 1, 1, 0, 2, 2, 1, 0, 1, 1, 2, 1, 2,
       1, 0, 1, 1, 2, 0, 0, 1, 2, 1, 1, 2, 2, 0, 0, 2, 0, 1, 0, 0, 2, 1,
       0, 1, 2, 2, 2, 2, 1, 1, 2, 0, 1, 1, 0, 0, 1, 1, 1, 2, 1, 1, 2, 0,
       2, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 2])

In [185]:
y_test

array([1, 1, 2, 1, 0, 0, 0, 0, 2, 0, 0, 0, 0, 1, 2, 2, 2, 1, 0, 2])

In [186]:
y_train_onehot = tf.one_hot(y_train, depth=3)

In [189]:
model = keras.Sequential([
    keras.layers.Dense(64, input_shape=(676,), activation='relu'),  # Ajusta 100 al número de características en X_combined
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(3, activation='softmax')  # 3 clases de prioridad
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train_onehot, epochs=10, batch_size=32)

Epoch 1/10
3/3 [==============================] - 2s 7ms/step - loss: 1.0780 - accuracy: 0.4125
Epoch 2/10
3/3 [==============================] - 0s 5ms/step - loss: 0.9462 - accuracy: 0.7375
Epoch 3/10
3/3 [==============================] - 0s 5ms/step - loss: 0.8512 - accuracy: 0.8500
Epoch 4/10
3/3 [==============================] - 0s 5ms/step - loss: 0.7445 - accuracy: 0.8875
Epoch 5/10
3/3 [==============================] - 0s 5ms/step - loss: 0.6560 - accuracy: 0.9125
Epoch 6/10
3/3 [==============================] - 0s 5ms/step - loss: 0.5658 - accuracy: 0.9500
Epoch 7/10
3/3 [==============================] - 0s 5ms/step - loss: 0.5011 - accuracy: 0.9375
Epoch 8/10
3/3 [==============================] - 0s 5ms/step - loss: 0.4199 - accuracy: 0.9625
Epoch 9/10
3/3 [==============================] - 0s 5ms/step - loss: 0.3565 - accuracy: 0.9750
Epoch 10/10
3/3 [==============================] - 0s 6ms/step - loss: 0.2961 - accuracy: 0.9750


In [192]:
y_pred = model.predict(X_test)

# Convertir las probabilidades en etiquetas predichas (la clase con la probabilidad más alta)
y_pred_classes = np.argmax(y_pred, axis=1)

# Calcular la precisión
accuracy = accuracy_score(y_test, y_pred_classes)

print(f'Precisión del modelo en el conjunto de prueba: {accuracy * 100:.2f}%')

1/1 [==============================] - 0s 20ms/step
Precisión del modelo en el conjunto de prueba: 80.00%


In [ ]:
"""model.save('Kai_correos.h5')"""